In [1]:
import json

import scipy as sp
import pandas as pd
import numpy as np

from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from collections import Counter

import lightgbm as lgb
np.random.seed(369)

from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm, tqdm_notebook

In [2]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = quadratic_weighted_kappa(y, X_p)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
def rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

### Image Features ###

In [3]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook
from keras.applications.densenet import preprocess_input, DenseNet121
import tensorflow as tf

train_df = pd.read_csv('input/train/train.csv')
img_size = 256
batch_size = 16

Using TensorFlow backend.


In [4]:
pet_ids = train_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

In [5]:
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

In [6]:
features = {}

def train_keras():
    from keras.models import Model
    from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
    from keras import backend as K
    import tensorflow as tf

    #https://regressionsessionsblog.wordpress.com/2018/06/11/stuck-on-an-issue-making-keras-predictions-in-parallel/
    #https://github.com/keras-team/keras/issues/4740
    jobs = 6
    config = tf.ConfigProto(intra_op_parallelism_threads=jobs, \
                            inter_op_parallelism_threads=jobs, \
                            allow_soft_placement=True, \
                            device_count = {'CPU': jobs})
    session = tf.Session(config=config)
    K.set_session(session)

    print("start")
    inp = Input((256,256,3))
    backbone = DenseNet121(input_tensor = inp, 
                            weights="input/densenet-keras/DenseNet-BC-121-32-no-top.h5",
                            include_top = False)
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
    x = AveragePooling1D(4)(x)
    out = Lambda(lambda x: x[:,:,0])(x)

    m = Model(inp,out)
    print("done")

    

    print("start keras prediction")

    for b in tqdm_notebook(range(n_batches)):
        start = b*batch_size
        end = (b+1)*batch_size
        batch_pets = pet_ids[start:end]
        batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
        for i,pet_id in enumerate(batch_pets):
            try:
                batch_images[i] = load_image("input/train/train_images/", pet_id)
            except:
                pass
        batch_preds = m.predict(batch_images)
        for i,pet_id in enumerate(batch_pets):
            features[pet_id] = batch_preds[i]
    print("done keras prediction")
    return m

In [7]:
from keras.models import Model
m = train_keras()

start
done
start keras prediction



done keras prediction


In [18]:
def save_json(features, filename="train_features"):
    with open(filename + ".json", 'w') as file:
        json.dump(features,file)
        #json.dump(dictionary, file, sort_keys=True, indent=4)
    return
def load_json(filename="file", dictionary=features):
    with open(filename + ".json", 'r') as file:
        loaded_file = json.load(file)
    return loaded_file

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(features, cls=NumpyEncoder)

with open("train_features.json", 'w') as f:
    json.dump(dumped, f)
    print("Saved train features json to disk")

In [32]:

#print(m.summary())

m.save_weights("keras_model_weights.h5")
print("Saved model weights to disk")
#with open("train_features.json", 'r') as f:
#    train_loaded_features = json.load(f)
#    train_loaded_features = json.loads(train_loaded_features)
#    print("loaded train features")
    

Saved model weights to disk


In [35]:
train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats.columns = ['pic_'+str(i) for i in range(train_feats.shape[1])]
test_df = pd.read_csv('input/test/test.csv')

pet_ids = test_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1
test_features = {}

def train_test_keras(m, n_batches):
    from keras.models import Model
    from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
    from keras import backend as K
    import tensorflow as tf

    #https://regressionsessionsblog.wordpress.com/2018/06/11/stuck-on-an-issue-making-keras-predictions-in-parallel/
    #https://github.com/keras-team/keras/issues/4740
    jobs = 6
    config = tf.ConfigProto(intra_op_parallelism_threads=jobs, \
                            inter_op_parallelism_threads=jobs, \
                            allow_soft_placement=True, \
                            device_count = {'CPU': jobs})
    session = tf.Session(config=config)
    K.set_session(session)

    print("start create model")
    inp = Input((256,256,3))
    backbone = DenseNet121(input_tensor = inp, 
                            weights="input/densenet-keras/DenseNet-BC-121-32-no-top.h5",
                            include_top = False)
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
    x = AveragePooling1D(4)(x)
    out = Lambda(lambda x: x[:,:,0])(x)

    m = Model(inp,out)
    print("done")
    
    print("start keras test prediction")

    for b in tqdm_notebook(range(n_batches)):
        start = b*batch_size
        end = (b+1)*batch_size
        batch_pets = pet_ids[start:end]
        batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
        for i,pet_id in enumerate(batch_pets):
            try:
                batch_images[i] = load_image("input/test/test_images/", pet_id)
            except:
                pass
        batch_preds = m.predict(batch_images)
        for i,pet_id in enumerate(batch_pets):
            test_features[pet_id] = batch_preds[i]
    print("done keras test prediction")
    return

train_test_keras(m, n_batches)

dumped = json.dumps(test_features, cls=NumpyEncoder)

with open("test_features.json", 'w') as f:
    json.dump(dumped, f)
    print("Saved test features json to disk")

start create model
done
start keras test prediction



done keras test prediction
Saved test features json to disk


In [43]:
test_feats = pd.DataFrame.from_dict(features, orient='index')
test_feats.columns = ['pic_'+str(i) for i in range(test_feats.shape[1])]
test_feats = test_feats.reset_index()
test_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

train_feats = train_feats.reset_index()
train_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

test_feats.head()

test_feats.to_csv(r'csv_out/img_test_feats_prediction_values.csv')
print("saved image test features prediction values to CSV")

saved image test features prediction values to CSV


In [167]:
train = pd.read_csv("input/train/train.csv")
test = pd.read_csv("input/test/test.csv")

target = train['AdoptionSpeed']
train_id = train['PetID']
test_id = test['PetID']

loaded_train_feats = {}
loaded_test_feats = {}
with open("train_features.json", 'r') as f:
    loaded_train_feats = json.load(f)
    loaded_train_feats = json.loads(loaded_train_feats)
    print("loaded train features")

with open("test_features.json", 'r') as f:
    loaded_test_feats = json.load(f)
    loaded_test_feats = json.loads(loaded_test_feats)
    print("loaded test features") 

print("creating dataframes from feature dictionaries")
train_feats = pd.DataFrame.from_dict(loaded_train_feats, orient='index')
train_feats.columns = ['pic_'+str(i) for i in range(train_feats.shape[1])]

test_feats = pd.DataFrame.from_dict(loaded_test_feats, orient='index')
test_feats.columns = ['pic_'+str(i) for i in range(test_feats.shape[1])]

train_feats = train_feats.reset_index()
train_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)
test_feats = test_feats.reset_index()
test_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

print("joining dataframes")
train = train.join(train_feats.set_index('PetID'),on='PetID')
test = test.join(test_feats.set_index('PetID'),on='PetID')
#train = pd.merge(train, train_feats, on = ['PetID'], how = 'left')
#test = pd.merge(test, test_feats, left_on = ['PetID'], right_on = ['PetID'], how = 'outer')

train.drop(['AdoptionSpeed', 'PetID'], axis=1, inplace=True)
test.drop(['PetID'], axis=1, inplace=True)

#prepared = train
#prepared.drop(['RescuerID', 'Name', 'Description', 'Vaccinated', 
#               'Color3', 'Dewormed', 'Health', 'VideoAmt', 'Type'], axis=1, inplace=True)

#train.drop(['RescuerID', 'Name', 'Description', 'Vaccinated', 
#               'Color3', 'Dewormed', 'Health', 'VideoAmt', 'Type'], axis=1, inplace=True)
#test.drop(['RescuerID', 'Name', 'Description', 'Vaccinated', 
#               'Color3', 'Dewormed', 'Health', 'VideoAmt', 'Type'], axis=1, inplace=True)

#prepared.to_csv(r'csv_out/cleaned_img_merge.csv')
#print("saved prepared csv")
train.to_csv(r'csv_out/train_csv_img_merge.csv')
print("saved train csv image merge")
test.to_csv(r'csv_out/test_csv_img_merge.csv')
print("saved test csv image merge")
print("done")

loaded train features
loaded test features
creating dataframes from feature dictionaries
joining dataframes
saved train csv image merge
saved test csv image merge
done


In [168]:
doc_sent_mag = []
doc_sent_score = []
nf_count = 0
print("loading sentiment train")
for petid in train_id:
    try:
        with open('input/train/train_sentiment/' + petid + '.json', 'r') as f:
            sentiment = json.load(f)
        doc_sent_mag.append(sentiment['documentSentiment']['magnitude'])
        doc_sent_score.append(sentiment['documentSentiment']['score'])
    except  Exception:
        nf_count += 1
        doc_sent_mag.append(-1)
        doc_sent_score.append(-1)
        pass

train.loc[:, 'doc_sent_mag'] = doc_sent_mag
train.loc[:, 'doc_sent_score'] = doc_sent_score

doc_sent_mag = []
doc_sent_score = []
nf_count = 0
print("loading sentiment test")
for petid in test_id:
    try:
        with open('input/test/test_sentiment/' + petid + '.json', 'r') as f:
            sentiment = json.load(f)
        doc_sent_mag.append(sentiment['documentSentiment']['magnitude'])
        doc_sent_score.append(sentiment['documentSentiment']['score'])
    except Exception:
        nf_count += 1
        doc_sent_mag.append(-1)
        doc_sent_score.append(-1)
        pass

test.loc[:, 'doc_sent_mag'] = doc_sent_mag
test.loc[:, 'doc_sent_score'] = doc_sent_score
print("done")

loading sentiment train
loading sentiment test
done


In [169]:
print("scikit training sentiment descriptions")
## WITHOUT ERROR FIXED
train_desc = train.Description.fillna("none").values
test_desc = test.Description.fillna("none").values

tfv = TfidfVectorizer(min_df=3,  max_features=10000,
        strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        stop_words = 'english')
    
# Fit TFIDF
tfv.fit(list(train_desc))
X =  tfv.transform(train_desc)
X_test = tfv.transform(test_desc)
print("X (tfidf):", X.shape)

svd = TruncatedSVD(n_components=200)
svd.fit(X)
# print(svd.explained_variance_ratio_.sum())
# print(svd.explained_variance_ratio_)
X = svd.transform(X)
print("X (svd):", X.shape)

X = pd.DataFrame(X, columns=['svd_{}'.format(i) for i in range(200)])
train = pd.concat((train, X), axis=1)
X_test = svd.transform(X_test)
X_test = pd.DataFrame(X_test, columns=['svd_{}'.format(i) for i in range(200)])
test = pd.concat((test, X_test), axis=1)

print("train:", train.shape)
print("done")

scikit training sentiment descriptions
X (tfidf): (14993, 10000)
X (svd): (14993, 200)
train: (14993, 480)
done


In [170]:
vertex_xs = []
vertex_ys = []
bounding_confidences = []
bounding_importance_fracs = []
dominant_blues = []
dominant_greens = []
dominant_reds = []
dominant_pixel_fracs = []
dominant_scores = []
label_descriptions = []
label_scores = []
nf_count = 0
nl_count = 0
print("loading metadata train")
for petid in train_id:
    try:
        with open('input/train/train_metadata/' + petid + '-1.json', 'r') as f:
            data = json.load(f)
            vertex_x = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['x']
            vertex_xs.append(vertex_x)
            vertex_y = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['y']
            vertex_ys.append(vertex_y)
            bounding_confidence = data['cropHintsAnnotation']['cropHints'][0]['confidence']
            bounding_confidences.append(bounding_confidence)
            bounding_importance_frac = data['cropHintsAnnotation']['cropHints'][0].get('importanceFraction', -1)
            bounding_importance_fracs.append(bounding_importance_frac)
            dominant_blue = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue']
            dominant_blues.append(dominant_blue)
            dominant_green = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green']
            dominant_greens.append(dominant_green)
            dominant_red = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red']
            dominant_reds.append(dominant_red)
            dominant_pixel_frac = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction']
            dominant_pixel_fracs.append(dominant_pixel_frac)
            dominant_score = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score']
            dominant_scores.append(dominant_score)
            if data.get('labelAnnotations'):
                label_description = data['labelAnnotations'][0]['description']
                label_descriptions.append(label_description)
                label_score = data['labelAnnotations'][0]['score']
                label_scores.append(label_score)
            else:
                nl_count += 1
                label_descriptions.append('nothing')
                label_scores.append(-1)
    except Exception:
        nf_count += 1
        vertex_xs.append(-1)
        vertex_ys.append(-1)
        bounding_confidences.append(-1)
        bounding_importance_fracs.append(-1)
        dominant_blues.append(-1)
        dominant_greens.append(-1)
        dominant_reds.append(-1)
        dominant_pixel_fracs.append(-1)
        dominant_scores.append(-1)
        label_descriptions.append('nothing')
        label_scores.append(-1)
        pass

print(nf_count)
print(nl_count)
train.loc[:, 'vertex_x'] = vertex_xs
train.loc[:, 'vertex_y'] = vertex_ys
train.loc[:, 'bounding_confidence'] = bounding_confidences
train.loc[:, 'bounding_importance'] = bounding_importance_fracs
train.loc[:, 'dominant_blue'] = dominant_blues
train.loc[:, 'dominant_green'] = dominant_greens
train.loc[:, 'dominant_red'] = dominant_reds
train.loc[:, 'dominant_pixel_frac'] = dominant_pixel_fracs
train.loc[:, 'dominant_score'] = dominant_scores
train.loc[:, 'label_description'] = label_descriptions
train.loc[:, 'label_score'] = label_scores


vertex_xs = []
vertex_ys = []
bounding_confidences = []
bounding_importance_fracs = []
dominant_blues = []
dominant_greens = []
dominant_reds = []
dominant_pixel_fracs = []
dominant_scores = []
label_descriptions = []
label_scores = []
nf_count = 0
nl_count = 0
print("loading metadata test")
for petid in test_id:
    try:
        with open('input/test/test_metadata/' + petid + '-1.json', 'r') as f:
            data = json.load(f)
            vertex_x = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['x']
            vertex_xs.append(vertex_x)
            vertex_y = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['y']
            vertex_ys.append(vertex_y)
            bounding_confidence = data['cropHintsAnnotation']['cropHints'][0]['confidence']
            bounding_confidences.append(bounding_confidence)
            bounding_importance_frac = data['cropHintsAnnotation']['cropHints'][0].get('importanceFraction', -1)
            bounding_importance_fracs.append(bounding_importance_frac)
            dominant_blue = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue']
            dominant_blues.append(dominant_blue)
            dominant_green = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green']
            dominant_greens.append(dominant_green)
            dominant_red = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red']
            dominant_reds.append(dominant_red)
            dominant_pixel_frac = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction']
            dominant_pixel_fracs.append(dominant_pixel_frac)
            dominant_score = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score']
            dominant_scores.append(dominant_score)
            if data.get('labelAnnotations'):
                label_description = data['labelAnnotations'][0]['description']
                label_descriptions.append(label_description)
                label_score = data['labelAnnotations'][0]['score']
                label_scores.append(label_score)
            else:
                nl_count += 1
                label_descriptions.append('nothing')
                label_scores.append(-1)
    except Exception:
        nf_count += 1
        vertex_xs.append(-1)
        vertex_ys.append(-1)
        bounding_confidences.append(-1)
        bounding_importance_fracs.append(-1)
        dominant_blues.append(-1)
        dominant_greens.append(-1)
        dominant_reds.append(-1)
        dominant_pixel_fracs.append(-1)
        dominant_scores.append(-1)
        label_descriptions.append('nothing')
        label_scores.append(-1)
        pass

print(nf_count)
test.loc[:, 'vertex_x'] = vertex_xs
test.loc[:, 'vertex_y'] = vertex_ys
test.loc[:, 'bounding_confidence'] = bounding_confidences
test.loc[:, 'bounding_importance'] = bounding_importance_fracs
test.loc[:, 'dominant_blue'] = dominant_blues
test.loc[:, 'dominant_green'] = dominant_greens
test.loc[:, 'dominant_red'] = dominant_reds
test.loc[:, 'dominant_pixel_frac'] = dominant_pixel_fracs
test.loc[:, 'dominant_score'] = dominant_scores
test.loc[:, 'label_description'] = label_descriptions
test.loc[:, 'label_score'] = label_scores
print("done")

loading metadata train
370
2
loading metadata test
137
done


In [171]:
train.drop(['Name', 'RescuerID', 'Description'], axis=1, inplace=True)
test.drop(['Name', 'RescuerID', 'Description'], axis=1, inplace=True)

train.drop(['dominant_green', 'doc_sent_score', 'FurLength', 'Vaccinated', 
               'Color3', 'Dewormed', 'Health', 'VideoAmt', 'Type',
           'bounding_importance', 'bounding_confidence', 'pic_79',
           'pic_252', 'pic_109', 'pic_197', 'pic_17', 'pic_104', 'pic_59'], axis=1, inplace=True)

In [172]:
numeric_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt', 'AdoptionSpeed', 
                'doc_sent_mag', 'doc_sent_score', 'dominant_score', 'dominant_pixel_frac', 
                'dominant_red', 'dominant_green', 'dominant_blue', 'bounding_importance', 
                'bounding_confidence', 'vertex_x', 'vertex_y', 'label_score'] +\
               [col for col in train.columns if col.startswith('pic') or col.startswith('svd')]
cat_cols = list(set(train.columns) - set(numeric_cols))
train.loc[:, cat_cols] = train[cat_cols].astype('category')
test.loc[:, cat_cols] = test[cat_cols].astype('category')
print(train.shape)
print(test.shape)

#print(test.head())
drop = "_drop100"
train.to_csv(r'csv_out/train_ims_merged' + drop + '.csv')
print("saved train csv image, metadata, sentiment merged")
test.to_csv(r'csv_out/test_ims_merged' + drop +'.csv')
print("saved test csv image, metadata, sentiment merged")
print("getting categorical features")
# get the categorical features
foo = train.dtypes
cat_feature_names = foo[foo == "category"]
cat_features = [train.columns.get_loc(c) for c in train.columns if c in cat_feature_names]
print("done")

(14993, 470)
(3948, 488)
saved train csv image, metadata, sentiment merged
saved test csv image, metadata, sentiment merged
getting categorical features
done


In [173]:
N_SPLITS = 5
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = StratifiedKFold(n_splits=N_SPLITS, random_state=42, shuffle=True)
    fold_splits = kf.split(train, target)
    cv_scores = []
    qwk_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0], N_SPLITS))
    all_coefficients = np.zeros((N_SPLITS, 4))
    feature_importance_df = pd.DataFrame()
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/' + str(N_SPLITS))
        if isinstance(train, pd.DataFrame):
            dev_X, val_X = train.iloc[dev_index], train.iloc[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        else:
            dev_X, val_X = train[dev_index], train[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y, importances, coefficients, qwk = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        all_coefficients[i-1, :] = coefficients
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            qwk_scores.append(qwk)
            print(label + ' cv score {}: RMSE {} QWK {}'.format(i, cv_score, qwk))
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = train.columns.values
        fold_importance_df['importance'] = importances
        fold_importance_df['fold'] = i
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)        
        i += 1
    print('{} cv RMSE scores : {}'.format(label, cv_scores))
    print('{} cv mean RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv QWK scores : {}'.format(label, qwk_scores))
    print('{} cv mean QWK score : {}'.format(label, np.mean(qwk_scores)))
    print('{} cv std QWK score : {}'.format(label, np.std(qwk_scores)))
    pred_full_test = pred_full_test / float(N_SPLITS)
    results = {'label': label,
               'train': pred_train, 'test': pred_full_test,
                'cv': cv_scores, 'qwk': qwk_scores,
               'importance': feature_importance_df,
               'coefficients': all_coefficients}
    return results

params = {'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 70,
          'max_depth': 9,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.02,
          'min_child_samples': 150,
          'min_child_weight': 0.02,
          'lambda_l2': 0.0475,
          'verbosity': -1,
          'data_random_seed': 17,
          'early_stop': 600,
          'verbose_eval': 100,
          'num_rounds': 10000}

def runLGB(train_X, train_y, test_X, test_y, test_X2, params):
    print('Prep LGB')
    d_train = lgb.Dataset(train_X, label=train_y)
    d_valid = lgb.Dataset(test_X, label=test_y)
    watchlist = [d_train, d_valid]
    print('Train LGB')
    num_rounds = params.pop('num_rounds')
    verbose_eval = params.pop('verbose_eval')
    early_stop = None
    if params.get('early_stop'):
        early_stop = params.pop('early_stop')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      categorical_feature=list(cat_features),
                      early_stopping_rounds=early_stop)
    
    print('Predict 1/2')
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    optR = OptimizedRounder()
    optR.fit(pred_test_y, test_y)
    coefficients = optR.coefficients()
    pred_test_y_k = optR.predict(pred_test_y, coefficients)
    print("Valid Counts = ", Counter(test_y))
    print("Predicted Counts = ", Counter(pred_test_y_k))
    print("Coefficients = ", coefficients)
    qwk = quadratic_weighted_kappa(test_y, pred_test_y_k)
    print("QWK = ", qwk)
    print('Predict 2/2')
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)
    return pred_test_y.reshape(-1, 1), pred_test_y2.reshape(-1, 1), model.feature_importance(), coefficients, qwk

print("Run LightGBM Train Model")
results = run_cv_model(train, test, target, runLGB, params, rmse, 'lgb')
print("done")

Run LightGBM Train Model
Started lgb fold 1/5
Prep LGB
Train LGB


C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 2, 3, 4, 5, 6, 7, 10, 468]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 600 rounds.
[100]	training's rmse: 1.03286	valid_1's rmse: 1.09873
[200]	training's rmse: 0.955043	valid_1's rmse: 1.07468
[300]	training's rmse: 0.903807	valid_1's rmse: 1.06484
[400]	training's rmse: 0.861022	valid_1's rmse: 1.05864
[500]	training's rmse: 0.823981	valid_1's rmse: 1.05426
[600]	training's rmse: 0.789444	valid_1's rmse: 1.0515
[700]	training's rmse: 0.75966	valid_1's rmse: 1.04855
[800]	training's rmse: 0.733438	valid_1's rmse: 1.04665
[900]	training's rmse: 0.70771	valid_1's rmse: 1.04505
[1000]	training's rmse: 0.683903	valid_1's rmse: 1.04393
[1100]	training's rmse: 0.661208	valid_1's rmse: 1.04279
[1200]	training's rmse: 0.639539	valid_1's rmse: 1.04231
[1300]	training's rmse: 0.620399	valid_1's rmse: 1.04157
[1400]	training's rmse: 0.601132	valid_1's rmse: 1.04113
[1500]	training's rmse: 0.581469	valid_1's rmse: 1.04053
[1600]	training's rmse: 0.563744	valid_1's rmse: 1.04
[1700]	training's rmse: 0.544759	valid_1'

C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 2, 3, 4, 5, 6, 7, 10, 468]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 600 rounds.
[100]	training's rmse: 1.03495	valid_1's rmse: 1.09145
[200]	training's rmse: 0.956729	valid_1's rmse: 1.06452
[300]	training's rmse: 0.902591	valid_1's rmse: 1.0536
[400]	training's rmse: 0.858616	valid_1's rmse: 1.0466
[500]	training's rmse: 0.822511	valid_1's rmse: 1.04222
[600]	training's rmse: 0.793378	valid_1's rmse: 1.03923
[700]	training's rmse: 0.764942	valid_1's rmse: 1.03683
[800]	training's rmse: 0.739484	valid_1's rmse: 1.03511
[900]	training's rmse: 0.714537	valid_1's rmse: 1.03389
[1000]	training's rmse: 0.689937	valid_1's rmse: 1.03294
[1100]	training's rmse: 0.666772	valid_1's rmse: 1.03214
[1200]	training's rmse: 0.644776	valid_1's rmse: 1.03145
[1300]	training's rmse: 0.624523	valid_1's rmse: 1.031
[1400]	training's rmse: 0.603369	valid_1's rmse: 1.03076
[1500]	training's rmse: 0.583592	valid_1's rmse: 1.03059
[1600]	training's rmse: 0.563707	valid_1's rmse: 1.02998
[1700]	training's rmse: 0.544721	valid_

...

Training until validation scores don't improve for 600 rounds.
[100]	training's rmse: 1.03566	valid_1's rmse: 1.08982
[200]	training's rmse: 0.958113	valid_1's rmse: 1.06248
[300]	training's rmse: 0.904145	valid_1's rmse: 1.04977
[400]	training's rmse: 0.863175	valid_1's rmse: 1.04221
[500]	training's rmse: 0.826946	valid_1's rmse: 1.03705
[600]	training's rmse: 0.793412	valid_1's rmse: 1.03361
[700]	training's rmse: 0.763222	valid_1's rmse: 1.03064
[800]	training's rmse: 0.734324	valid_1's rmse: 1.02827
[900]	training's rmse: 0.710221	valid_1's rmse: 1.02675
[1000]	training's rmse: 0.685177	valid_1's rmse: 1.02545
[1100]	training's rmse: 0.661229	valid_1's rmse: 1.02475
[1200]	training's rmse: 0.638601	valid_1's rmse: 1.02418
[1300]	training's rmse: 0.617182	valid_1's rmse: 1.02386
[1400]	training's rmse: 0.596695	valid_1's rmse: 1.02333
[1500]	training's rmse: 0.578004	valid_1's rmse: 1.02311
[1600]	training's rmse: 0.560371	valid_1's rmse: 1.02309
[1700]	training's rmse: 0.541886	va

C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 2, 3, 4, 5, 6, 7, 10, 468]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 600 rounds.
[100]	training's rmse: 1.03626	valid_1's rmse: 1.09318
[200]	training's rmse: 0.958158	valid_1's rmse: 1.0648
[300]	training's rmse: 0.903699	valid_1's rmse: 1.05246
[400]	training's rmse: 0.861604	valid_1's rmse: 1.04644
[500]	training's rmse: 0.82393	valid_1's rmse: 1.04213
[600]	training's rmse: 0.7906	valid_1's rmse: 1.03846
[700]	training's rmse: 0.760295	valid_1's rmse: 1.03671
[800]	training's rmse: 0.732531	valid_1's rmse: 1.03508
[900]	training's rmse: 0.705178	valid_1's rmse: 1.03415
[1000]	training's rmse: 0.681793	valid_1's rmse: 1.03368
[1100]	training's rmse: 0.659112	valid_1's rmse: 1.03304
[1200]	training's rmse: 0.636603	valid_1's rmse: 1.03269
[1300]	training's rmse: 0.615344	valid_1's rmse: 1.0324
[1400]	training's rmse: 0.597488	valid_1's rmse: 1.0326
[1500]	training's rmse: 0.577913	valid_1's rmse: 1.03257
[1600]	training's rmse: 0.559492	valid_1's rmse: 1.03242
[1700]	training's rmse: 0.543076	valid_1'

C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 2, 3, 4, 5, 6, 7, 10, 468]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 600 rounds.
[100]	training's rmse: 1.03512	valid_1's rmse: 1.09475
[200]	training's rmse: 0.956667	valid_1's rmse: 1.06621
[300]	training's rmse: 0.90111	valid_1's rmse: 1.05368
[400]	training's rmse: 0.855444	valid_1's rmse: 1.04577
[500]	training's rmse: 0.816423	valid_1's rmse: 1.04166
[600]	training's rmse: 0.782636	valid_1's rmse: 1.03861
[700]	training's rmse: 0.75324	valid_1's rmse: 1.0366
[800]	training's rmse: 0.725484	valid_1's rmse: 1.0348
[900]	training's rmse: 0.700621	valid_1's rmse: 1.0332
[1000]	training's rmse: 0.677416	valid_1's rmse: 1.03222
[1100]	training's rmse: 0.654773	valid_1's rmse: 1.03168
[1200]	training's rmse: 0.633067	valid_1's rmse: 1.0311
[1300]	training's rmse: 0.612844	valid_1's rmse: 1.03068
[1400]	training's rmse: 0.59392	valid_1's rmse: 1.0301
[1500]	training's rmse: 0.574021	valid_1's rmse: 1.03001
[1600]	training's rmse: 0.557851	valid_1's rmse: 1.02972
[1700]	training's rmse: 0.542437	valid_1's 

In [176]:
imports = results['importance'].groupby('feature')['feature', 'importance'].mean().reset_index()
imports.sort_values('importance', ascending=False)

,feature,importance
1,Breed1,1655.4
0,Age,739.2
2,Breed2,493.8
10,State,443.6
18,label_score,366.2
363,svd_184,353.2
410,svd_47,353.0
343,svd_166,350.2
9,Quantity,346.2
436,svd_70,339.2


In [177]:
print("saving out feature importance list to CSV")
imports.to_csv("csv_out/features_importance.csv")
print("done")

saving out feature importance list to CSV
done


In [178]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84
train_predictions = [r[0] for r in results['train']]
train_predictions = optR.predict(train_predictions, coefficients_).astype(int)
Counter(train_predictions)

[0.50197585 1.92066053 2.50625882 2.93203965]


Counter({3: 2919, 1: 3036, 2: 4012, 4: 4284, 0: 742})

In [179]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84
test_predictions = [r[0] for r in results['test']]
test_predictions = optR.predict(test_predictions, coefficients_).astype(int)
Counter(test_predictions)

[0.50197585 1.92066053 2.50625882 2.93203965]


Counter({2: 2163, 3: 1282, 4: 101, 1: 398, 0: 4})

In [180]:
print("True Distribution:")
print(pd.value_counts(target, normalize=True).sort_index())
print("Test Predicted Distribution:")
print(pd.value_counts(test_predictions, normalize=True).sort_index())
print("Train Predicted Distribution:")
print(pd.value_counts(train_predictions, normalize=True).sort_index())

True Distribution:
0    0.027346
1    0.206096
2    0.269259
3    0.217368
4    0.279931
Name: AdoptionSpeed, dtype: float64
Test Predicted Distribution:
0    0.001013
1    0.100811
2    0.547872
3    0.324721
4    0.025583
dtype: float64
Train Predicted Distribution:
0    0.049490
1    0.202494
2    0.267592
3    0.194691
4    0.285733
dtype: float64


In [181]:
pd.DataFrame(sk_cmatrix(target, train_predictions), index=list(range(5)), columns=list(range(5)))

,0,1,2,3,4
0,81,122,98,57,52
1,358,1077,937,441,277
2,210,1011,1317,821,678
3,78,601,998,758,824
4,15,225,662,842,2453


In [182]:
quadratic_weighted_kappa(target, train_predictions)
rmse(target, [r[0] for r in results['train']])
submission = pd.DataFrame({'PetID': test_id, 'AdoptionSpeed': test_predictions})
submission.head()

,PetID,AdoptionSpeed
0,378fcc4fc,2
1,73c10e136,2
2,72000c4c5,2
3,e147a4b9f,3
4,43fbba852,3


In [183]:
submission.to_csv('submissions/LightBGM with image features/submission.csv', index=False)